In [1]:
from os.path import join, basename, splitext
import json
from glob import glob
from tqdm import tqdm
from itertools import islice
import numpy as np

from common import tokenize
from nltk.tokenize import sent_tokenize
import logging

from IPython.display import Markdown, display, HTML
from common import get_all_docs, word_tokenizer

logger = logging.getLogger(__name__)
DATA_FOLDER = '../data/'

Using TensorFlow backend.


In [2]:
all_docs = get_all_docs(DATA_FOLDER)
val_docs = glob(join(DATA_FOLDER, 'validate/*.txt'))

with open(join(DATA_FOLDER, 'best.json'), 'r') as f:
    best = json.load(f)
    
with open(join(DATA_FOLDER, 'gold.json'), 'r') as f:
    gold = json.load(f)

In [3]:
def hprint(string, i=1):
    display(HTML('<h%s>%s</h%s>' % (i, string, i)))

def print_file(sub_name, domain, i=1, tokenize=False):    
    found = [fn for fn in domain if sub_name in fn]
    for fn in found:
        hprint(fn + '\n', i)
        with open(fn, 'r') as f:
            text = f.read()
            if tokenize:
                print(tokenize(text))
            else:
                print(text)
#         !cat {'\''+fn+'\''}

def inspect_result(qname, start_index=0, end_index=2):
    true_docs = gold[qname]

    hprint('Query\n')
    print_file(qname, val_docs, 5)
    for i, tname in enumerate(true_docs):
        hprint('True %s\n' %(i+1))
        print_file(tname, all_docs, 5)

    for i, bname in enumerate(best[qname][start_index:end_index]):
        hprint('Found %s\n' % (i+1))
        print_file(bname, all_docs, 5)    

In [4]:
val_docs_ids = set([splitext(basename(f))[0] for f in val_docs])
validation_ids = [k for k in gold.keys() if k in val_docs_ids]
print('validation set size %s' % len(validation_ids))
for qname in validation_ids:
    print(qname)

validation set size 130
_40_1a_2011126928A10010101RU
_df_5c_2010114200A10010101RU
_ab_68_2006139128A10010101RU
_1b_60_2010122997A10010101RU
_ef_e0_2010102239A10010101RU
_a5_03_2009140669A10010101RU
_87_7b_2010120096A10010101RU
_c3_b8_2009111341A10010101RU
_ff_96_2010138984A10010101RU
_be_ef_2010152978A10010101RU
_59_59_2009131618A10010101RU
_87_6d_2008148713A10010101RU
_0c_89_2009110686A10010101RU
_03_43_2008151996A10010101RU
_94_b1_2009122096A10010101RU
_e6_cc_2009147572A10010101RU
_18_b9_2010101690A10010101RU
_79_fe_2010107007A10010101RU
_ae_fc_2010107173A10010101RU
_8d_51_2010136608A10010101RU
_05_7a_2007148915A10010101RU
_82_1b_2010112743A10010101RU
_91_ca_2010129617A10010101RU
_3f_fb_2010139538A10010101RU
_23_fc_2011105696A10010101RU
_b6_9c_2010104075A10010101RU
_f0_fa_2010100979A10010101RU
_14_9d_2011108876A10010101RU
_a8_27_2010114657A10010101RU
_c9_ad_2010109635A10010101RU
_c0_79_2009148052A10010101RU
_33_40_2009146768A10010101RU
_88_8b_2011124782A10010101RU
_8f_ca_2011124783A1

# Характеристика качества данных

| Заявка | Порочащий д-т без описания   | Хорошее совпадение назаний|
|------|------|
|   _40_1a_2011126928A10010101RU  | 1||
|   _df_5c_2010114200A10010101RU  | 1|1|

# Подход

Так как весь текст патента избыточен для выяснения конкретного новшества данного автента, то имеет смысл брать куски текста вокруг ключевых слов, указывающих на само новшество. Ниже пример таких слов.

Весь текст патента избыточен еще и потому что суть изобретения может быть применима в разных областях, например "кулинария" (подогрев плиты) и "одежда"(подогрев одежды). 

Поиск по ключевым словам:
- из названия патента
- заявлять
- заключать в том , что
- суть
- новый
- применение в качество
- новый технический результат заключаться в возможность
- новый технологичный способ получение
- преимущество
- сущность изобретение
- эффективный метод, средство, способ
- удалось
- задача , на решение который направлять настоящий изобретение
- новизна
- предмет изобретение
- описывать способ получение
- технический результат полезный модель заключаться
    
Можно дополнить этот список наиболее близкими словами к данным, полученных на этом корпусе.
    
Имеет смысл разметить в тексте патентов такие куски, которые указывают на саму суть и обучаться на двух трех предложениях вокруг ключевых слов. Таким образом есть вероятность что мы лучше поймем отличия патентов.

Метод - CNN, которые выделяют наиболее значимые куски текста, делают саммаризацию в режиме unsupervised learning.

Простой способ индикации описания сути изобретения - векторное среднее движущегося n-gramm по тексту. Нужно пройтись так по всему корпусу для каждого предложения, получить векторные представления n-gramm(фраз-индикаторов), взять n ближайших соседий от заранее заданных фраз-индикаторов(по типу тех что приведены выше) - это и будут фразы-индикаторы. При этом можно будет придумывать новые фразы и смотреть, попадают ли они в этот кластер. Также из n-gramm можно будет выкидывать слова, которые не имеют целевой значимости, например:

"РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула".


In [5]:
import gensim

model = gensim.models.Word2Vec.load(join(DATA_FOLDER, 'vectors/old/w2v_model_300'))
wv = model.wv

In [47]:
wv.most_similar(positive=['результат'], topn=10)

[('получать', 0.666301429271698),
 ('предлагать', 0.6384239792823792),
 ('неудачн', 0.6303017735481262),
 ('заключаться', 0.6248302459716797),
 ('изобретение', 0.597603440284729),
 ('обеспечивать', 0.584223747253418),
 ('использование', 0.5819722414016724),
 ('существенный', 0.579962432384491),
 ('достигаться', 0.5796698927879333),
 ('достигаемый', 0.5791462659835815)]

In [110]:
def most_probable(wv, words, n=4, enable_logging=True):
    if len(words) <= n:
        return words

    wv.init_sims()
    
    used_words = set([word for word in words if word in wv])
    if len(used_words) != len(words):
        ignored_words = set(words) - set(used_words)
        if enable_logging:
            logger.warning("vectors for words %s are not present in the model, ignoring these words", ignored_words)
    if not used_words:
        print(words)
        raise ValueError("cannot select a word from an empty list")
    vectors = np.vstack(wv.word_vec(word, use_norm=True) for word in used_words).astype(np.float32)
    mean = gensim.matutils.unitvec(vectors.mean(axis=0)).astype(np.float32)
    dists = np.dot(vectors, mean)
    most = sorted(zip(dists, used_words), reverse=True)[:n]
    
    return [word for sim, word in most]

In [8]:
sent = 'РЕФЕРАТ описывать способ получение ВНУТРЕННИЙ ГИДРОФТОРОЛЕФИН формула'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['описывать', 'получение', 'формула']


In [50]:
sent = 'технический результат полезный модель заключаться'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['заключаться', 'результат', 'технический']


In [10]:
sent = 'новый технологичный способ получение'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

final n-gram: ['способ', 'получение', 'новый']


#### Теперь придумаем новую фразу

In [11]:
sent = 'новизна данный изобретение в том что гидроусилитель статический'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'в'} are not present in the model, ignoring these words


final n-gram: ['данный', 'новизна', 'изобретение']


Здесь важно натренировать w2v так чтобы в корпусе присутствовали все слова кроме стоп слов, например, если убрать "данный", то получаем:

In [12]:
sent = 'новизна изобретение в том что гидроусилитель статический '
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'в'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'изобретение', 'статический']


А это уже хуже. Поэтому надо брать достаточное скользящее окно, например, 5 и в нем выбирать 3 наиболее вероятных терма. Также плохо брать слишком большое окно

In [13]:
sent = 'новизна изобретение в том что гидроусилитель статический с повышенный уровень отказоустойчивость'
words = sent.lower().split()
splited = most_probable(wv, words, n=3)

print('final n-gram: %s' % splited)

vectors for words {'что', 'том', 'с', 'в'} are not present in the model, ignoring these words


final n-gram: ['новизна', 'отказоустойчивость', 'уровень']


# Реализация

In [27]:
def iconv(sent, n=3):
    "Returns a sliding window (of width n) over data from the iterable"
    "   s -> (s0,s1,...s[n-1]), (s1,s2,...,sn), ...                   "
    it = iter(sent)
    result = tuple(islice(it, n))
    if len(result) == n:
        yield result    
    for elem in it:
        result = result[1:] + (elem,)
        yield result

In [78]:
def convolv_on_doc(doc, window=5, n=3):
    """
    doc - list object with sentences as lists of words
    window - windows size
    n - n-gram size
    """
    for sent in doc:
        step = max(n, min(window, len(sent)))
        for ngram in iconv(sent, step):
            yield ngram

**Explanation**

In [79]:
doc = [[1,2],[3,4,5],[6,7,8,9],[10,11,12,13,14],[15,16,17,18,19,20]]
for tup in convolv_on_doc(doc):
    print(tup)

(3, 4, 5)
(6, 7, 8, 9)
(10, 11, 12, 13, 14)
(15, 16, 17, 18, 19)
(16, 17, 18, 19, 20)


In [82]:
from nltk.corpus import stopwords

stop_list = stopwords.words('russian')
stop_list.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на', 'ко'])
stop_list = set(stop_list)


fn = '../data/validate/_ab_68_2006139128A10010101RU.txt'
with open(fn, 'r') as f:
    doc_text = f.read()

sents = sent_tokenize(doc_text)

doc = [tokenize(s, stop_list) for s in sents]

In [108]:
def get_convolved_ngrams(doc, window=5, n=3):
    result = []
    for ngram in convolv_on_doc(doc, window, n):
        probable_ngram = most_probable(wv, ngram, n, enable_logging=False)
        avg_ngram = np.mean(wv[probable_ngram], axis=0)
        result.append((probable_ngram, avg_ngram))
    return result

In [111]:
convolved_doc = get_convolved_ngrams(doc, window=5, n=3)

('crc', 'handbook', 'of', 'chemistry', 'and')


ValueError: cannot select a word from an empty list

In [101]:
len(convolved_doc)

1201

# Пример

In [44]:
inspect_result('_0c_89_2009110686A10010101RU', 0,3)

﻿способ выравнивание бетон и растворный смесь на горизонтальный поверхность .
 изобретение относиться к область строительство и мочь быть использовать для укладка бетон и растворный смесь на горизонтальный поверхность , например , плита монолитный покрытие .
 технический задача на решение который направлять данный изобретение являться упрощение процесс выравнивание бетонный и растворный смесь , повышение качество и производительность выполнять работа , экономия расходный и отделочный материал .
 технический результат достигаться в способ выравнивание бетон и растворный смесь на горизонтальный поверхность установка маяк и выравнивание осуществляться с использование виброрейка , установленный на направлять по уровень маяк сплошной слой . .
 способ выравнивание бетон и растворный смесь на горизонтальный поверхность , включать установка маяк , отличать тем , что выравнивание осуществлять с использование виброрейка , установленный на направлять по уровень маяк сплошной слой .
 изобретение о

способ изготовление маяк для выравнивать стяжка пол и устройство для осуществление данный способ  . 
 изобретение относиться к строительство и мочь быть использовать для выполнение выравнивать стяжка пол .
 технический результат изобретение являться : минимизация отклонение плоскость поверхность маяк от задавать уровень , повышение быстродействие проведение работа при изготовление маяк для выравнивать стяжка пол .
 способ изготовление маяк для выравнивать стяжка пол включать очистка поверхность основание под стяжка с возможность размещение слой гидроизоляция и разметка пространство для определение место расположение маяк .
 далее устанавливать вдоль линия разметка формовать приспособление для укладка раствор .
 затем на заготовка маяк в вид отформованный раствор устанавливать срезать устройство , определять и устанавливать горизонтальный плоскость поперечный срез заготовка с помощь пузырьковый уровень и два винт регулировка станина , выставлять с помощь лазерный нивелир необходимый вер

способ выравнивание бетон и растворный смесь на горизонтальный поверхность  . 
 способ выравнивание бетон и растворный смесь на горизонтальный поверхность , включать установка маяк , отличать тем , что выравнивание осуществлять с использование виброрейка , установленный на направлять по уровень маяк сплошной слой .  .   .   .


маяк для выравнивание уровень поверхность  . 
 полезный модель относиться к строительство , а именно к инструмент , использовать при отделочный работа , и предназначать для выравнивание уровень поверхность при оштукатуривание .
 технический результат полезный модель заключаться в создание маяк для выравнивание уровень поверхность , позволять минимизировать толщина выравнивать слой , универсальный , удобный , простой и наименее затратный в производство и использование .
 технический результат достигаться тем , что верхний часть профиль маяк для выравнивание уровень поверхность в поперечный разрез иметь вид выпуклый дуга , а так же тем , что толщина материал , из который он изготовлять составлять 1 мм и более .  . 
 1 .
 маяк для выравнивание уровень поверхность , отличать тем , что о профиль в поперечный разрез иметь вид выпуклый дуга , крайний точка который опираться на выравнивать поверхность .
 2 .
 маяк для выравнивание уровень поверхность по п.1 , отличать тем , что толщина материа

способ изготовление струнный маяк из строительный раствор  . 
 способ изготовление струнный маяк из строительный раствор , включать в себя крепление кронштейн к оштукатуривать плоскость , натягивание на они нить , провеска плоскость , устройство маячный марка как основа будущий маяк и устройство маяк из строительный раствор , отличать тем , что нить заменять на струна из стальной проволока или тонкий трос и использовать на все время оштукатуривание плоскость в качество косвенный арматура будущее маяк , вершина который составлять струна , опирать на твердеть строительный раствор , который набрасывать на натягивать в искомый плоскость струна с " горка " , а лишний , выступать за искомый плоскость , раствор срезать при переход строительный раствор из жидкий состояние в твердый фаза .  .   .   .
